# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 26 2022 10:39AM,242739,19,ACG-DESTRUCTION,ACG North America LLC,Released
1,Jul 26 2022 10:24AM,242736,10,8325420,MedStone Pharma LLC,Released
2,Jul 26 2022 10:24AM,242736,10,8325421,MedStone Pharma LLC,Released
3,Jul 26 2022 10:24AM,242736,10,8325422,MedStone Pharma LLC,Released
4,Jul 26 2022 10:24AM,242736,10,8325423,MedStone Pharma LLC,Released
5,Jul 26 2022 10:24AM,242736,10,8325425,MedStone Pharma LLC,Released
6,Jul 26 2022 10:20AM,242730,15,PNC377603A,"Person & Covey, Inc.",Released
7,Jul 26 2022 10:20AM,242730,15,PNC377611A,"Person & Covey, Inc.",Released
8,Jul 26 2022 10:20AM,242730,15,PNC377612A,"Person & Covey, Inc.",Released
9,Jul 26 2022 10:20AM,242730,15,PNC377615A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,242732,Released,1
35,242733,Released,2
36,242734,Executing,1
37,242736,Released,5
38,242739,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242732,NaN,NaN,1.0
242733,NaN,NaN,2.0
242734,NaN,1.0,NaN
242736,NaN,NaN,5.0
242739,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242581,0.0,0.0,1.0
242638,0.0,1.0,0.0
242643,0.0,0.0,1.0
242685,0.0,1.0,0.0
242694,0.0,11.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242581,0,0,1
242638,0,1,0
242643,0,0,1
242685,0,1,0
242694,0,11,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242581,0,0,1
1,242638,0,1,0
2,242643,0,0,1
3,242685,0,1,0
4,242694,0,11,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242581,,,1
1,242638,,1,
2,242643,,,1
3,242685,,1,
4,242694,,11,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 26 2022 10:39AM,242739,19,ACG North America LLC
1,Jul 26 2022 10:24AM,242736,10,MedStone Pharma LLC
6,Jul 26 2022 10:20AM,242730,15,"Person & Covey, Inc."
15,Jul 26 2022 10:18AM,242734,19,"AdvaGen Pharma, Ltd"
16,Jul 26 2022 10:15AM,242731,15,"Virtus Pharmaceuticals, LLC"
45,Jul 26 2022 10:13AM,242733,15,"Mizner Bioscience, LLC"
47,Jul 26 2022 10:12AM,242732,15,"Carwin Pharmaceutical Associates, LLC"
48,Jul 26 2022 10:07AM,242729,15,"Brookfield Pharmaceuticals, LLC"
87,Jul 26 2022 10:02AM,242728,15,"Alliance Pharma, Inc."
108,Jul 26 2022 9:49AM,242725,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 26 2022 10:39AM,242739,19,ACG North America LLC,,,1
1,Jul 26 2022 10:24AM,242736,10,MedStone Pharma LLC,,,5
2,Jul 26 2022 10:20AM,242730,15,"Person & Covey, Inc.",,,9
3,Jul 26 2022 10:18AM,242734,19,"AdvaGen Pharma, Ltd",,1,
4,Jul 26 2022 10:15AM,242731,15,"Virtus Pharmaceuticals, LLC",,17,12
5,Jul 26 2022 10:13AM,242733,15,"Mizner Bioscience, LLC",,,2
6,Jul 26 2022 10:12AM,242732,15,"Carwin Pharmaceutical Associates, LLC",,,1
7,Jul 26 2022 10:07AM,242729,15,"Brookfield Pharmaceuticals, LLC",,7,32
8,Jul 26 2022 10:02AM,242728,15,"Alliance Pharma, Inc.",,,21
9,Jul 26 2022 9:49AM,242725,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 10:39AM,242739,19,ACG North America LLC,1,,
1,Jul 26 2022 10:24AM,242736,10,MedStone Pharma LLC,5,,
2,Jul 26 2022 10:20AM,242730,15,"Person & Covey, Inc.",9,,
3,Jul 26 2022 10:18AM,242734,19,"AdvaGen Pharma, Ltd",,1,
4,Jul 26 2022 10:15AM,242731,15,"Virtus Pharmaceuticals, LLC",12,17,
5,Jul 26 2022 10:13AM,242733,15,"Mizner Bioscience, LLC",2,,
6,Jul 26 2022 10:12AM,242732,15,"Carwin Pharmaceutical Associates, LLC",1,,
7,Jul 26 2022 10:07AM,242729,15,"Brookfield Pharmaceuticals, LLC",32,7,
8,Jul 26 2022 10:02AM,242728,15,"Alliance Pharma, Inc.",21,,
9,Jul 26 2022 9:49AM,242725,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 10:39AM,242739,19,ACG North America LLC,1,,
1,Jul 26 2022 10:24AM,242736,10,MedStone Pharma LLC,5,,
2,Jul 26 2022 10:20AM,242730,15,"Person & Covey, Inc.",9,,
3,Jul 26 2022 10:18AM,242734,19,"AdvaGen Pharma, Ltd",,1,
4,Jul 26 2022 10:15AM,242731,15,"Virtus Pharmaceuticals, LLC",12,17,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 10:39AM,242739,19,ACG North America LLC,1.0,NaN,NaN
1,Jul 26 2022 10:24AM,242736,10,MedStone Pharma LLC,5.0,NaN,NaN
2,Jul 26 2022 10:20AM,242730,15,"Person & Covey, Inc.",9.0,NaN,NaN
3,Jul 26 2022 10:18AM,242734,19,"AdvaGen Pharma, Ltd",NaN,1.0,NaN
4,Jul 26 2022 10:15AM,242731,15,"Virtus Pharmaceuticals, LLC",12.0,17.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 10:39AM,242739,19,ACG North America LLC,1.0,0.0,0.0
1,Jul 26 2022 10:24AM,242736,10,MedStone Pharma LLC,5.0,0.0,0.0
2,Jul 26 2022 10:20AM,242730,15,"Person & Covey, Inc.",9.0,0.0,0.0
3,Jul 26 2022 10:18AM,242734,19,"AdvaGen Pharma, Ltd",0.0,1.0,0.0
4,Jul 26 2022 10:15AM,242731,15,"Virtus Pharmaceuticals, LLC",12.0,17.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2669586,42.0,20.0,45.0
15,1456383,77.0,24.0,0.0
16,485442,14.0,14.0,0.0
18,242685,0.0,1.0,0.0
19,485473,1.0,1.0,0.0
20,1699012,129.0,53.0,0.0
21,242705,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2669586,42.0,20.0,45.0
1,15,1456383,77.0,24.0,0.0
2,16,485442,14.0,14.0,0.0
3,18,242685,0.0,1.0,0.0
4,19,485473,1.0,1.0,0.0
5,20,1699012,129.0,53.0,0.0
6,21,242705,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42.0,20.0,45.0
1,15,77.0,24.0,0.0
2,16,14.0,14.0,0.0
3,18,0.0,1.0,0.0
4,19,1.0,1.0,0.0
5,20,129.0,53.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,15,Released,77.0
2,16,Released,14.0
3,18,Released,0.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,45.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,20.0,24.0,14.0,1.0,1.0,53.0,0.0
Released,42.0,77.0,14.0,0.0,1.0,129.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,45.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,20.0,24.0,14.0,1.0,1.0,53.0,0.0
2,Released,42.0,77.0,14.0,0.0,1.0,129.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,45.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,20.0,24.0,14.0,1.0,1.0,53.0,0.0
2,Released,42.0,77.0,14.0,0.0,1.0,129.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()